# Guide to using the Opentrons OT2 for BJS Biotechnologies

By Adam Zaini



***

## *Table of Contents*

0. Opening chapter
1. Preparing the robot for the MDx Sequence
2. Components of the MDx Sequence
3. Full sequence test and algorithm
4. Creating custom labwares, commands, and procedures
5. Examples, tests, and common loops
6. Xxplate for the 54 wells and 96 wells



________________________________________________________________________________________________________________________________


## 0. *Opening Chapter*

### (i) *Objective and contents*

This document contains the code for MDx sequence for the 24 well xxplate. On top of that, it also provides guides, tutorials, and examples to assist the user in using the Opentrons OT2 for the purpose of the xxpress machine by BJS Biotechnologies. To see the full sequence for the 24 well plate without markdowns, search for a file titled '24 well xxplate (video sequence)'. In this example, only 4 DNA samples were utilised, this is due to the limitations of the heating block only being able to accomodate 4 vials at a time.

### (ii) *Opentrons Python API Documentation*

Before continuing with the notebook, it is recommended that the documentation on the Python API for the Opentrons OT2 robot is read. The information available on the document would help the reader understand this notebook better.

The document can be found here: https://docs.opentrons.com/index.html

Although it is recommended to read the entire document, the main sections of the document to be read are:
1. Labware                  (https://docs.opentrons.com/labware.html)
2. Creating a pipette       (https://docs.opentrons.com/pipettes.html)
3. Atomic liquid handling   (https://docs.opentrons.com/atomic%20commands.html)


***


## 1. *Preparing the robot for the MDx Sequence*

### (i) *Connecting the code to the robot's hardware*

Run the cell below to connect the current notebook to the robot's hardware. The cell should only be run once upon starting the robot or restarting the kernel. Running the code twice will result in errors.

In [ ]:
# THIS CODE SHOULD ONLY BE RUN ONCE UPON STARTING THE ROBOT / RESTARTING THE KERNEL
from opentrons import robot
robot.connect()

### (ii) *Resetting the robot and clearing command history*

Run the cell below to clear command history and reset the robot.

In [ ]:
from opentrons import robot
robot.clear_commands()          #Clears command history
robot.reset()                   #Resets the robot

#Running this cell twice in a row will cause errors

These two commands are typically used when a new process is required. On Opentron, new commands are dependent on all the commands executed prior to the new command. Hence, the function

is useful for when the user wants to execute a command independent of the previous commands. For example, a tip has been picked up by the pipette from well 'A1'. Let's say you manually remove the tip and place it back by hand into well 'A1'. If you command the robot to pick up a tip from well 'A1', the robot will return an error as its command history shows that a tip is already placed on the pipette and/or a tip has already been taken from well 'A1'. Hence, clearing the command history will allow you to pick up a tip from 'A1' again.

Resetting the robot will clear of all the labware and instruments that has been loaded into the robot. This is a necesarry step to take when the user wishes to change the labware or position of the labware. The new set or position of the labware and instruments should then be loaded again.

### (iii) *Loading the labware and instruments*

Run the cell below to load all labware and instruments into the robot. This should be the first cell of code the user will execute after connecting the code to the robot's hardware.

In [ ]:
from opentrons import instruments, robot, labware

# labware and instruments involved
tiprack10 = labware.load('tiprack-10ul','4')             #10ul tip rack
tiprack300 = labware.load('opentrons-tiprack-300ul','1')  #300ul tip rack

pipette1 = instruments.P10_Single(mount='left')           #assign 10ul tip to left mount
pipette2 = instruments.P300_Single(mount='right')         #assign 300ul tip to right mount

vialrack1 = labware.load('BJS_vialrack','2')             #BJS vial rack
sample = labware.load('trough-12row','6')                 #12 row trough
xxplate = labware.load('xxpress_24well','3')              #24 well xxplate
vialrack2 = labware.load('Micronic_vialrack','7')         #Micronic vial rack
heatingblock = labware.load('heating_block','5')          #Heating block
vialrack3 = labware.load('Micronic_vialrack','8')

To load the labware, the user would have to use the function that takes the following format:

The slots available are slots 1 to 11 as seen on the deck of the robot. The slot on the top right is reserved for trash. To see the full list of labware equipments and their names, run the following code:

In [ ]:
from opentrons import labware
labware.list()

Loading instruments such as pipette uses the following format:

The pipette names can be found on the labware list as well. The left and right mount are reserved for a group of tip sizes. In  this case, the left mount, which is the smaller mount, is reserved for a 10ul pipette tip where as the right mount is reserved for a 300ul tip.

**Notes:**
* Only rows B and C (B1-B12, C1-C12) of the 300ul tiprack by GEB can be used. This is because Opentrons has their own 300ul tiprack which has different dimensions. Because of the way the hardware and software of the robot interacts with a tiprack, we are unable to create our own tiprack.
* Only wells A1, A2, B1 and B2 of the heating block can be used. This is due the heating block being hand made and asymmetrical. Other wells can be accessed by tweaking the offset coordinate values by trial and error. This step is explained further in 4. Creating custom labwares, commands, and procedures.
* All custom labwares should use custom functions that accomodates for the offset instead of using basic Opentrons functions.For example, use fill_xxplate('A1', 5, '10') instead of pipette10.dispense(5, plate.wells('A1')). This is further explained in the following step 1.(iv) Defining functions and 4.(i) Creating custom commands.
* In this notebook, from chapter 1 to 5 all examples uses the xxplate with **24 wells**. The functions for the xxplate with 54 wells and 96 wells can be found in chapter 6.

### (iv) *Definining functions*


Since custom labwares are used for the MDx sequence, custom functions have to be defined to be used together with these custom labwares. There are 3 functions defined, each of which is a way to interact with a well of the xxplate or a BJS vial. The 5 functions and its purpose are:

Each of these functions consist of three arguments that has to be satisfied.

e.g - To fill well D6 of the xxplate by 5ul using a 10ul tip, you would write

Run the following cell the define the functions required to execute the sequences.

In [ ]:
#######################################################################

# Function to fill xxplate
def fill_xxplate(well, vol, tip):
    if tip == '10':
        offset = xxplate.wells('A1').from_center(x=15.2,y=10,z=0.2)            #offset of the custom labware
        destination = (xxplate.wells(well), offset)                            #destination for the pipette (specified well)
        pipette1.move_to(destination, strategy='arc')                          #move pipette to destination
        pipette1.dispense(vol)                                                 #dispense specified volume into well
        print(well, 'has been filled by', vol, 'ul')
    elif tip == '300':
        if vol < 81:
            offset = xxplate.wells('A1').from_center(x=15.9,y=8.9,z=0.2)        #offset of the custom labware
            destination = (xxplate.wells(well), offset)                         #destination for the pipette (specified well)
            pipette2.move_to(destination, strategy='arc')                       #move pipette to destination
            pipette2.dispense(vol)                                              #dispense specified volume into well
            print(well, 'has been filled by', vol, 'ul')
        else:
            print('Well is not able to hold the specified volume.')
    else:
        print('Please select a valid tip(\'10\',\'300\')')
    return

#######################################################################

# Function to fill a vial on the BJS vial rack
def fill_BJSvial(well, vol, tip):
    if tip == '10':
        if vol < 11:                                                        
            offset = vialrack1.wells('A1').from_center(x=3.2,y=1.5,z=0)     #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette1.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette1.dispense(vol)                                          #dispense specified volume into vial
            print(well, 'has been filled by', vol, 'ul')
        else:
            print('Tip is not able to hold the liquid volume specified.')
    elif tip == '300':
        if vol < 301:
            offset = vialrack1.wells('A1').from_center(x=3.75,y=0.8,z=0)    #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette2.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette2.dispense(vol)                                          #dispense specified volume into vial
            print(well, 'has been filled by', vol, 'ul')
        else:
            print('Tip is not able to hold the liquid volume specified.')
    else:
        print('Please specify a valid tip (\'10\',\'300\')')
    return

#######################################################################

# Function to extract from a vial on the BJS vial rack
def extract_BJSvial(well, vol, tip):
    if tip == '10':
        if vol < 11:
            offset = vialrack1.wells('A1').from_center(x=3.2,y=1.5,z=-0.8)  #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette1.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette1.aspirate(vol)                                          #extract specified volume from vial
            print(vol, 'ul has been extracted from', well)
        else:
            print('Volume to be extracted is greater than tip size. Please select a lower volume or a different tip')
    elif tip == '300':
        if vol < 301:
            offset = vialrack1.wells('A1').from_center(x=3.75,y=0.8,z=-0.8) #offset of the custom labware
            destination = (vialrack1.wells(well), offset)                   #destination for the pipette (specified well)
            pipette2.move_to(destination, strategy='arc')                   #move pipette to destination
            pipette2.aspirate(vol)                                          #extract specified volume from vial
            print(vol, 'ul has been extracted from', well)
        else:
            print('Volume to be extracted is greater than tip size. Please select a lower volume')
    else:
        print('Please specify a valid tip (\'10\',\'300\')')        
    return

#######################################################################

# Function to fill a vial on the Micronic vial rack
def fill_micronic(well, vol, tip):
    if tip == '10':
        offset = vialrack2.wells('A1').from_center(x=3.3,y=3.0,z=0.0)    #10ul tip
        destination = (vialrack2.wells(well), offset)
        pipette1.move_to(destination, strategy='arc')
        pipette1.dispense(vol)
        print(well, 'has been filled by', vol, 'ul')
    elif tip == '300':
        offset = vialrack2.wells('A1').from_center(x=3.8,y=2.3,z=0.0)    #300ul tip
        destination = (vialrack2.wells(well), offset)
        pipette2.move_to(destination, strategy='arc')
        pipette2.dispense(vol)
        print(well, 'has been filled by', vol, 'ul')
    return

#######################################################################

# Function to extract from a vial on the Micronic vial rack
def extract_micronic(well,vol):
    offset = vialrack3.wells('A1').from_center(x=3.8,y=2.3,z=-0.7)    #300ul tip
    destination = (vialrack3.wells(well), offset)
    pipette2.move_to(destination, strategy='arc')
    pipette2.aspirate(vol)
    print(vol, 'ul has been extracted from', well)
    return

#######################################################################



***

## 2. *Components of the MDx Sequence*

The MDx procedure consist of 5 sequences that takes place in the following order. The 5 sequences are:

1. DNA Sample Extraction
2. Transporting Vials
3. Rehydration Buffer
4. Preload Control
5. Supernatant Extraction


### (i) *DNA Sample Extraction Sequence*

This process transfers 40ul of liquid from a micronic vial rack cotnaining the DNA sample to a micronic vial rack containing the extraction buffer.

In [ ]:
%%time

vol = 40
rows = ['A','B','C','D']
int = 1
for r in rows:
    tip = 'B' + str(int)
    well_aspirate = r + '1'
    well_dispense = r + '12'
    pipette2.pick_up_tip(tiprack300.wells(tip))
    extract_micronic(well_aspirate, vol)
    fill_micronic(well_dispense, vol, '300')
    pipette2.drop_tip()
    int += 1

### (ii) *Transporting Vials*

This section consist of two steps; transferring vials from a Micronic vial rack to the heating block, and from the heating block to the BJS vial rack. It uses an adaptor that utilises a horizontal force to attach and detach from the vials.

**NOTE**: For this process, the robot has to pick up a 'virtual' 10ul pipette tip

In [ ]:
%%time

micronic2heater('A12', 'B2')
micronic2heater('B12', 'A2')
micronic2heater('C12', 'B1')
micronic2heater('D12', 'A1')

In [ ]:
%%time

heater2vialrack('B2','A12')
heater2vialrack('A2','C12')
heater2vialrack('B1','E12')
heater2vialrack('A1','G12')

pipette1.drop_tip()

### (iii) *Rehydration Buffer Sequence*

This process uses a 300ul tip. Since the same solution will be dispensed into every well, a single pipette can be used. The code is time optimised by reducing the number of movements the robot would have to make depending on the volume specified by the user. It utilises for loops to iterate over all rows and columns of the xxplate.

In [ ]:
def buffer_sequence(vol):
    if 0<vol<26:                                                 #FOR THE CASE WHEN VOL IS 1 - 25
        to_extract = vol*12                                      #aspirate 12 times of the specified
        pipette2.aspirate(to_extract, sample.wells('A1'))        #aspirate
        rows = ['A','B']                                         #rows to fill first
        for r in rows:                                           #iterate over row and columns
            for c in range (1,7):
                well = r + str(c)                                #define well
                fill_xxplate(well, vol, '300')                   #fill xxplate
        rows = ['C','D']                                         #next rows to fill
        pipette2.aspirate(to_extract, sample.wells('A1'))        #aspirate
        for r in rows:                                           #iterate over rows and columns
            for c in range (1,7):
                well = r + str(c)                                #define well
                fill_xxplate(well, vol, '300')                   #fill xxplate
    elif 25<vol<51:                                              #FOR THE CASE WHEN VOL IS 26 - 50
        to_extract = vol*6
        rows = ['A','B','C','D']
        for r in rows:
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range (1,7):
                well = r + str(c)
                fill_xxplate(well, vol, '300')
    elif 50<vol<81:                                              #FOR THE CASE WHEN VOL IS 51 - 80
        to_extract = vol*3
        rows = ['A','B','C','D']
        for r in rows:
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range(1,4):
                well = r + str(c)
                fill_xxplate(well, vol, '300')
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range (4,7):
                well = r + str(c)
                fill_xxplate(well, vol, '300')
    else:
        print('Please specifiy a volume between 1ul and 80ul')
    return

In [ ]:
%%time

pipette2.pick_up_tip(tiprack300.wells('C1'))   #Pick up tip
buffer_sequence(25)                            #Call the function to start the sequence
pipette2.drop_tip()                            #Drop the tip

### (iv) *Preload Control Sequence*

This step uses a 300ul tip to fill the xxplate with 3 types of preload control; a positive, a negative, and a zero. With each preload control a different tip is used. Each preload control will occupy two columns on the xxplate.

In [ ]:
vol = 20
rows = ['A','B','C','D']
init = 1

pipette2.pick_up_tip(tiprack300.wells('B6'))
pipette2.aspirate(vol*8, sample.wells('A2'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 3

pipette2.pick_up_tip(tiprack300.wells('B7'))
pipette2.aspirate(vol*8, sample.wells('A3'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 5

pipette2.pick_up_tip(tiprack300.wells('B8'))
pipette2.aspirate(vol*8, sample.wells('A4'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

### (v) *Supernatant Extraction Sequence Test*

This process uses a 300ul tip to extract from each of the 4 DNA samples to spread across the xxplate. Each row of the xxplate contains each of the DNA samples.

In [ ]:
vol = 6
rows_xx = ['A','B','C','D']
rows = ['A','C','E','G']
init = 9
for i in range (0,4):
    tip = 'B' + str(init)
    pipette2.pick_up_tip(tiprack300.wells(tip))
    well_aspirate = rows[i] + '12'
    extract_BJSvial(well_aspirate, vol*6, '300')
    for n in range (1,7):
        well_dispense = rows_xx[i] + str(n)
        fill_xxplate(well_dispense, vol, '300')
    pipette2.drop_tip()
    init += 1


________________________________________________________________________________________________________________________________


## 3. *Full Procedure Test and Algorithm*

The full procedure is a compilation of all 5 sequences of the MDx. The structure of the full procedure is as shown in the cell below. Running the cell would run the full sequence.

In [ ]:
%%time

# DNA SAMPLE SEQUENCE (uses tips B1 to B12 - 300ul)
DNAsample_sequence(100)

pipette1.pick_up_tip(tiprack10.wells('H11'))    #pick up virtual pipette

pipette1.delay(seconds = 10)  # ATTACH THE ADAPTOR DURING THE 10s DELAY


# DNA Extraction
rows = ['A','B','C','D']
int = 1
for r in rows:
    tip = 'B' + str(int)
    well_aspirate = r + '1'
    well_dispense = r + '12'
    pipette2.pick_up_tip(tiprack300.wells(tip))
    extract_micronic(well_aspirate, 40)
    fill_micronic(well_dispense, 40, '300')
    pipette2.drop_tip()
    int += 1

#Transfer vials from micronic vial rack to heating block    
micronic2heater('A12', 'B2')
micronic2heater('B12', 'A2')
micronic2heater('C12', 'B1')
micronic2heater('D12', 'A1')

#Rehydration Buffer
pipette2.pick_up_tip(tiprack300.wells('B5'))
buffer_sequence(25)
pipette2.drop_tip()

#Preload control
vol = 20
rows = ['A','B','C','D']
init = 1

pipette2.pick_up_tip(tiprack300.wells('B6'))
pipette2.aspirate(vol*8, sample.wells('A2'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 3

pipette2.pick_up_tip(tiprack300.wells('B7'))
pipette2.aspirate(vol*8, sample.wells('A3'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

init = 5

pipette2.pick_up_tip(tiprack300.wells('B8'))
pipette2.aspirate(vol*8, sample.wells('A4'))
for r in rows:
    well = r + str(init)
    fill_xxplate(well, vol, '300')
    well2 = r + str(init+1)
    fill_xxplate(well2, vol, '300')
        
pipette2.drop_tip()

#Transfer vials from heating block to vial rack
heater2vialrack('B2','A12')
heater2vialrack('A2','C12')
heater2vialrack('B1','E12')
heater2vialrack('A1','G12')

#Supernatant
vol = 6
rows_xx = ['A','B','C','D']
rows = ['A','C','E','G']
init = 9
for i in range (0,4):
    tip = 'B' + str(init)
    pipette2.pick_up_tip(tiprack300.wells(tip))
    well_aspirate = rows[i] + '12'
    extract_BJSvial(well_aspirate, vol*6, '300')
    for n in range (1,7):
        well_dispense = rows_xx[i] + str(n)
        fill_xxplate(well_dispense, vol, '300')
    pipette2.drop_tip()
    init += 1

robot.home()

The algorithm to run the MDx procedure is as follows:

1. Connect the code to the robot
2. Load the labware and instruments
3. Define the functions (e.g - fill_xxplate() )
4. Call the sequence functions in the order you would like them to take place
5. Home the robot
6. Clear commands and reset history to prepare for next sequence

To see the full code without markdown cells, search for a notebook titled '24 well Sequence' in the robot's Jupyter Notebook.



***


## 4. *Creating custom labwares, commands, and procedure*

This section focuses on helping the user creating their own sequences and labwares.

### *Making custom labwares*

To make custom labwares, the user will have to call the labware.create() function. The if statement is to ensure a labwares with the same name are not repeated within the labware list. The official document to making custom labwares can be found here: https://docs.opentrons.com/labware.html 

In [ ]:
plate_name = 'BJS_vialrack'
if plate_name not in labware.list():
    custom_plate = labware.create(
        plate_name,
        grid = (12,8),
        spacing = (9.1,9.3),
        diameter = 6.4,
        depth = 42,
        volume = 1100)
else:
    print('Custom labware failed to create. Labware name is already in the labware list.')

The function contains 6 arguments that we need to satisfy. The arguments are shown below.

To check if the labware has been succesfully created. Run the following code to check the list of labwares. The labware name should be in the list.

In [ ]:
from opentrons import labware
labware.list

To delete a custom labware from the list, run the following code. Be careful when deleting labwares as **all** measurements will be lost. Make a note of the measurements before deleteing.

In [ ]:
from opentrons.data_storage import database
database.delete_container('labware_name')

### *Making custom commands*

To make custom commands for a custom labware. We have to consider the offset of the labware equipment. The offset is the distance between the bottom left corner of the labware with the most bottom left well of the labware. Unfortunately, when using the labware.create() function, we are not able to define an offset. Hence, the robot assumes an offset for us. This offset tends to be wrong. We may fix this by considering the offset in our custom commands.

In [ ]:
#EXAMPLE CODE
def custom_command(well, volume):
    offset = custom_labware.wells('A1').from_center(x = 2, y = 2, z = 2) # Consider the offset (obtained by trial & error)
    destination = (custom_labware.wells(well), offset)                   # Set destination = well to visit + offset
    pipette.move_to(destination, strategy = 'arc')                       # Move to destination using an 'arc' strategy
    pipette.dispense(volume)                                             # Dispense
    return

The function defined above is tasked to move to a defined well on a custom labware to dispense a specified volume. The offset is obtained by manually adjusting the x, y, z values and observing the position of the pipette on the physical labware.

**NOTE**: When adjusting x, y, z values, it is advised to use a positive z value (e.g - z = 2) to ensure it does not collide with anything on the labware deck.

The destination is then set to the well + the offset, which should be the actual position of your well on the labware. The next command instructs the pipette to move to the destination. And the next command dispenses a defined volume of liquid from its current position.

### *Making custom sequences and procedures*

Custom procedures are requried to automate a process. There are two main ways to create a procedure:

**NOTE:** When testing your sequences, it is best to use print() before using the physical functions. This helps avoid any accidents and ensure you are confident with executing your sequence.

In [ ]:
#EXAMPLE 1 - Sequential
def sequential(vol):
    print('Pick up tip A1')
    print('Extract', vol, 'ul from sample A1')
    print('Dispense', vol, 'ul in vial A1')
    print('Drop tip')
    print('Pick up tip A2')
    print('Extract', vol, 'ul from sample A1')
    print('Dispense', vol, 'ul in vial A2')
    print('Drop tip')
    print('Pick up tip B1')
    print('Extract', vol, 'ul from sample A1')
    print('Dispense', vol, 'ul in vial B1')
    print('Drop tip')
    print('Pick up tip B2')
    print('Extract', vol, 'ul from sample A1')
    print('Dispense', vol, 'ul in vial B2')
    print('Drop tip')
    return

sequential(10)

In [ ]:
#EXAMPLE 2 - Looping

def looping(rows, cols, vol):
    init = 1
    tiprow = 'A'
    for r in rows:
        for c in range(1, cols+1):
            tip = tiprow + str(init)
            vial = r + str(c)
            print('Pick up tip', tip)
            print('Extract', vol, 'ul from sample A1')
            print('Dispense', vol, 'ul in vial', vial)
            print('Drop tip')
            init += 1
            if init == 3:
                init = 1
                tiprow = 'B'
    return

looping(['A','B'], 2, 10)

From examples 1 and 2, we can see that both produces similar results. The sequential example has more flexiblity and is simpler to understand, however, the looping example is far more scalable and efficient. 

For the looping sequence, if the user wish to increase the number of rows, the user would have to simply add more rows and columns into the argument. The other would require you to add more lines of code and make a change to each line. The sequential sequence has the advantage of flexibility and simplicity. For example, if the user wishes to fill wells that are in a random order (e.g - A1, B4, C2, D6), the sequential structure would be easier to program in this case. 

All in all, both produces the same results. Which structure the user wishes to adapt depends on the process and the capabilities of the user.



________________________________________________________________________________________________________________________________


## 5. *Examples, tests, and common loops*

Below are some examples and test that the user can use to learn from. It also contains some common loops. This helps to show how to use for loops to iterate over the grid of the labware.

### (i) *Filling one well of the xxplate*

In [ ]:
pipette1.pick_up_tip(tiprack10.wells('A1'))   #pick up a 10ul tip
pipette1.aspirate(10, sample.wells('A1'))     #extract 10ul from sample
fill_xxplate('A1', 10, '10')                  #fill specified well on xxplate by a specified volume
pipette1.drop_tip()                           #drop tip in the trash

### (ii) *Filling one vial on the BJS vial rack*

In [ ]:
pipette1.pick_up_tip(tiprack10.wells('A2'))
pipette1.aspirate(10, sample.wells('A2'))
fill_BJSvial('A1', 10, '10')
pipette1.drop_tip()

### (iii) *Extracting from one vial on the BJS vial rack*

In [ ]:
pipette1.pick_up_tip(tiprack10.wells('A1'))
extract_BJSvial('A1', 10, '10')
fill_BJSvial('A2', 10, '10')
pipette1.drop_tip()

### (iv) *Filling a column of BJS Vial with 300ul liquid*

In [ ]:
rows = ['A','B','C','D','E','F','G','H']            #define rows
pipette2.pick_up_tip(tiprack300.wells('D1'))        #pick up a tip
#Iterate over the rows
for r in rows:
    well = r + '1'                                  #define the well to dispense to
    pipette2.aspirate(300, sample.wells('A1'))      #aspirate from the sample
    fill_BJSvial(well, 300, '300')                  #fill the vial
pipette2.drop_tip()                                 #drop tip

### (v) *Loop to fill all wells  of the xxplate using one tip*

In [ ]:
pipette1.pick_up_tip(tiprack10.wells('A1'))         #pick up a 10ul tip
rows = ['A','B','C','D']                            #define existing rows
for c in range (1,7):                               #iterate over all rows and columns  
    for r in rows:
        well = r + str(c)                           #define well in current iteration
        pipette1.aspirate(6, sample.wells('A1'))    #extract specified volume from sample
        fill_xxplate(well, 6, '10')                 #fill specified well on the xxplate by a specified amount
pipette1.drop_tip()
print('xxplate has been filled')

### (vi) *Loop to fill all wells of the xxplate using separate tip for each well*

In [ ]:
init = 1                                            #Start the tip sequence from Column 1 of the tip rack
tiprow = 'A'                                        #Start the tip sequence from Row A of the tip rack
rows = ['A','B','C','D']                            #Define existing rows of the xxplate

for r in rows:                                      #Iterate over 4 rows of the 24 well xxplate
    for c in range (1,7):                           #Iterate over 6 columns of the 24 well xxplate
        tip = tiprow + str(init)                    #Define the tip to pick up
        well = r + str(c)                           #Define well to fill in the xxplate
        pipette1.pick_up_tip(tiprack10.wells(tip))  #Pick up 10ul tip from the tiprack
        pipette1.aspirate(6, sample.wells('A1'))    #Extract a specified volume from a specified well of the sample
        fill_xxplate(well, 6, '10')                 #Fill a specified well on the xxplate by a specified volume
        pipette1.drop_tip()                         #Drop current tip in the trash
        init += 1                                   #Use the following column of the tip rack for the next iteration
        if init == 13:                              #Maximum number of columns in the tip rack is 12, hence no 13th column
            init = 1                                #Reset back to first column of the tip rack
            tiprow = 'B'                            #Use Row B of the tip rack

print('xxplate has been filled')

### (vii) *Function that has the rows and columns as arguments to iterate over*

In [ ]:
def iteration(rows,columns,vol):
    for r in rows:                           #iterate over the rows defined
        for c in (1,columns+1):              #iterate over each column (+1 due to pythons number sequence)
            well = r + str(c)                #define well to pick up (e.g - 'A1')
            fill_xxplate(well, vol, '300')   #call function to fill a well of the xxplate
    return

An example of using this function is shown in the cell below:

In [ ]:
pipette2.pick_up_tip(tiprack300.wells('A1')) #pick up a tip
iteration(['A','B','C'], 2, 25)              #call the function
pipette2.drop_tip()                          #drop tip

## 6. *Xxplate for 54 wells and 96 wells*

All the examples prior to this has been using the 24 well xxplate. To use the 54 well or 96 well xxplate, be sure to remember to clear the command history, reset the robot, and load the new labwares into the robot. You can find the name for the 54 wells and 96 wells xxplate by calling the function labware.list().

### (i) *54 wells xxplate*

The following cell defines the function for the robot to interact with the 54 plate xxplate. This is done by making a custom command that accomodates the offset of the 54 plate xxplate. The coordinates of the offset is deduced using trial and error. The function can be used to create custom procedures similar to the ones done for the 24 well xxplate.

In [ ]:
def fill_xxplate54(well, vol, tip):
    if tip == '10':
        offset = xxplate.wells('A1').from_center(x=18.7,y=12,z=0.0)
        destination = (xxplate.wells(well), offset)
        pipette1.move_to(destination, strategy='arc')
        pipette1.dispense(vol)
    elif tip == '300':
        offset = xxplate.wells('A1').from_center(x=19.5,y=10.5,z=0.2)
        destination = (xxplate.wells(well), offset)
        pipette2.move_to(destination, strategy='arc')
        pipette2.dispense(vol)
    else:
        print('Please specify a valid tip (\'10\' or \'300\')')
    return

### (ii) *96 well xxplate*

The following cell defines the function for the robot to interact with the 96 plate xxplate. This is done by making a custom command that accomodates the offset of the 96 plate xxplate. The coordinates of the offset is deduced using trial and error. The function can be used to create custom procedures similar to the ones done for the 24 well xxplate. 

In [ ]:
def fill_xxplate96(well, vol, tip):
    if tip == '10':
        offset = xxplate.wells('A1').from_center(x=24.5,y=15.5,z=0.0)
        destination = (xxplate.wells(well), offset)
        pipette1.move_to(destination, strategy='arc')
    elif tip == '300':
        offset = xxplate.wells('A1').from_center(x=25.7,y=13.5,z=0.2)
        destination = (xxplate.wells(well), offset)
        pipette2.move_to(destination, strategy='arc')
    else:
        print('Please specify a valid tip (\'10\' or \'300\')')
    return

### (iii) *Buffer sequence (54 wells)*

The following cell shows the function that calls the sequence to fill a 54 well xxplate with the rehyration buffer. It is similar to the code for the 24 well xxplate where it is time optimised for the volume the user wishes to dispense into a well. In this sequence, a single tip is used to extract from a sample and dispensed into every well of the xxplate.

In [ ]:
def buffer_sequence(vol):
    if 16.66 < vol < 33.33:
        rows = ['A','B','C','D','E','F']
        to_extract = vol*9
        for r in rows:
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
    elif 11.11 < vol < 16.67:
        rows = ['A','B']
        to_extract = vol*18
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
        rows = ['C','D']
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
        rows = ['E','F']
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
    elif 5.55 < vol < 11.12:
        rows = ['A','B','C']
        to_extract = vol*27
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
        rows = ['D','E','F']
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
    elif 0 < vol < 5.56:
        rows = ['A','B','C','D','E','F']
        to_extract = vol*54
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,10):
                well = r + str(c)
                fill_xxplate54(well, vol, '300')
    else:
        print('Please specify a volume within the valid range')
    return

### (iv) *Buffer sequence (96 wells)*

The following cell shows the function that calls the sequence to fill a 96 well xxplate with the rehyration buffer. It is similar to the code for the 24 well xxplate where it is time optimised for the volume the user wishes to dispense into a well. In this sequence, a single tip is used to extract from a sample and dispensed into every well of the xxplate.

In [ ]:
def buffer_sequence(vol):
    if 12.5 < vol < 22.6:
        rows = ['A','B','C','D','E','F','G','H']
        to_extract = vol*12
        for r in rows:
            pipette2.aspirate(to_extract, sample.wells('A1'))
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
    elif 6.25 < vol < 12.51:
        rows = ['A','B']
        to_extract = vol*24
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
        rows = ['C','D']
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
        rows = ['E','F']
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
        rows = ['G','H'http://10.44.187.13:48888/notebooks/Instruction%20Manual.ipynb#]
        pipette2,aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
    elif 3.12 < vol < 6.26:
        rows = ['A','B','C','D']
        to_extract = vol*48
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
        rows = ['E','F','G','H']
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')
    elif 0 < vol < 3.13:
        rows = ['A','B','C','D','E','F','G']
        to_extract = vol*96
        pipette2.aspirate(to_extract, sample.wells('A1'))
        for r in rows:
            for c in range (1,13):
                well = r + str(c)
                fill_xxplate96(well, vol, '300')                
    else:
        print('Please specify a volume within the valid range')
    return